In [1]:
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import demoji
import html
from datetime import datetime

In [2]:
# Load the data
df = pd.read_csv("D:\kaimtenx\project\week4\Amharic-Ecommerce-Data-Extractor\data\cleaned_data.csv")


In [4]:
# Load the data
df = pd.read_csv("D:\kaimtenx\project\week4\Amharic-Ecommerce-Data-Extractor\data\cleaned_data.csv")
display(df.head())


,channel,date,message,views,sender_id,media_type,tokens
0,@Fashiontera,2025-06-14 13:32:36+00:00,〰️〰️〰️〰️〰️〰️〰️\nMade in Vietnam \n🔸🔸🔸🔸🔸🔸\nSize...,1192,-1001175527648,photo,"['〰️〰️〰️〰️〰️〰️〰️', 'Made', 'in', 'Vietnam', '🔸..."
1,@Fashiontera,2025-06-13 19:55:32+00:00,〰️〰️〰️〰️〰️〰️〰️\nAirforce \nMade in Vietnam \n🔸...,1098,-1001175527648,photo,"['〰️〰️〰️〰️〰️〰️〰️', 'Airforce', 'Made', 'in', '..."
2,@Fashiontera,2025-06-11 20:11:16+00:00,〰️〰️〰️〰️〰️〰️〰️\nUnder Armur\nMade in Vietnam \...,1117,-1001175527648,photo,"['〰️〰️〰️〰️〰️〰️〰️', 'Under', 'Armur', 'Made', '..."
3,@Fashiontera,2025-06-04 19:18:01+00:00,〰️〰️〰️〰️〰️〰️〰️\nNB\nMade in Vietnam \n🔸🔸🔸🔸🔸🔸\n...,1835,-1001175527648,photo,"['〰️〰️〰️〰️〰️〰️〰️', 'NB', 'Made', 'in', 'Vietna..."
4,@Fashiontera,2025-06-03 19:34:21+00:00,〰️〰️〰️〰️〰️〰️〰️\nHigh Quality shoes \n🔸🔸🔸🔸🔸🔸\nS...,1622,-1001175527648,photo,"['〰️〰️〰️〰️〰️〰️〰️', 'High', 'Quality', 'shoes',..."


In [5]:
#display
(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14453 entries, 0 to 14452
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   channel     14453 non-null  object
 1   date        14453 non-null  object
 2   message     14453 non-null  object
 3   views       14453 non-null  int64 
 4   sender_id   14453 non-null  int64 
 5   media_type  14291 non-null  object
 6   tokens      14453 non-null  object
dtypes: int64(2), object(5)
memory usage: 790.5+ KB


In [6]:
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

In [8]:
#text cleaning function
def clean_text(text):
    """Complete text cleaning pipeline"""
    if pd.isna(text):
        return np.nan
        # Convert HTML entities
    text = html.unescape(text)
        # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
        # Remove emojis
    text = demoji.replace(text, '')
        # Remove URLs and Telegram mentions
    text = re.sub(r'http\S+|www\S+|t\.me/\S+|@\w+', '', text)
        # Remove decorative characters (like 〰️)
    text = re.sub(r'[〰️🔸]+', ' ', text)
        # Remove extra newlines and normalize whitespace
    text = re.sub(r'\n+', ' ', text)
    text = ' '.join(text.split())
    # Normalize Amharic orthographic variants
    norm_map = {
        '[ሐኀ]': 'ሀ',
        '[አኣ]': 'አ',
        '[ጸፀ]': 'ጸ',
    }
    for pattern, repl in norm_map.items():
        text = re.sub(pattern, repl, text)

    # Remove foreign chars except Ethiopic letters/digits/punctuations
    text = re.sub(r"[^\u1200-\u137F0-9፡።፣፤፥፦፧፨ \n]", '', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()
# Apply cleaning
df['clean_message'] = df['message'].apply(clean_text)

C:\Users\ayish\AppData\Local\Temp\ipykernel_2180\1768192006.py:9: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  text = BeautifulSoup(text, 'html.parser').get_text()


In [9]:
#tokinization
def tokenize_text(text):
    """Tokenize text while handling special cases"""
    if pd.isna(text):
        return []
    
    # Handle common product patterns
    text = re.sub(r'(\bMade in \w+)', r'\1', text)  # Protect "Made in Vietnam"
    text = re.sub(r'(\bSize [A-Z0-9]+\b)', r'\1', text)  # Protect "Size XL"
    
    # Basic tokenization
    tokens = re.findall(r'\b[\w-]+\b', text)
    
    return tokens

# Apply tokenization
df['tokens'] = df['clean_message'].apply(tokenize_text)

In [10]:
print(df[['clean_message', 'tokens']].head())

                                       clean_message  \
0  4142 3400 ስልክ 251945355266 ፋሽን ተራ አድራሻ አዲስ አበባ...   
1  4041424344 4500 ስልክ 251945355266 ፋሽን ተራ አድራሻ አ...   
2  4142 3500 ስልክ 251945355266 ፋሽን ተራ አድራሻ አዲስ አበባ...   
3  4243 2900 ስልክ 251945355266 ፋሽን ተራ አድራሻ አዲስ አበባ...   
4  44 3500 ስልክ 251945355266 ፋሽን ተራ አድራሻ አዲስ አበባ ጦ...   

                                              tokens  
0  [4142, 3400, ስልክ, 251945355266, ፋሽን, ተራ, አድራሻ,...  
1  [4041424344, 4500, ስልክ, 251945355266, ፋሽን, ተራ,...  
2  [4142, 3500, ስልክ, 251945355266, ፋሽን, ተራ, አድራሻ,...  
3  [4243, 2900, ስልክ, 251945355266, ፋሽን, ተራ, አድራሻ,...  
4  [44, 3500, ስልክ, 251945355266, ፋሽን, ተራ, አድራሻ, አ...  


In [11]:
#Entity Extraction
def extract_entities(tokens):
    """Identify products, brands, and attributes"""
    entities = {
        'brands': [],
        'attributes': [],
        'materials': []
    }
        # Common brand names in fashion
    fashion_brands = {'nike', 'adidas', 'under armour', 'new balance', 'puma', 'armur'}
        # Common product attributes
    attributes = {'size', 'made in', 'high quality', 'color', 'xl', 'xxl'}
        # Join tokens for multi-word matching
    text = ' '.join(tokens).lower()
        # Extract brands
    for brand in fashion_brands:
        if brand in text:
            entities['brands'].append(brand)
    
    # Extract attributes
    for attr in attributes:
        if attr in text:
            entities['attributes'].append(attr)
    
    # Extract materials (simple heuristic)
    materials = {'leather', 'cotton', 'polyester', 'vietnam'}
    for token in tokens:
        if token.lower() in materials:
            entities['materials'].append(token.lower())
    
    return entities

# Apply entity extraction
df['entities'] = df['tokens'].apply(extract_entities)

# Create expanded columns
df['brands'] = df['entities'].apply(lambda x: '|'.join(x['brands']))
df['attributes'] = df['entities'].apply(lambda x: '|'.join(x['attributes']))
df['materials'] = df['entities'].apply(lambda x: '|'.join(x['materials']))

In [12]:
#metadata processing
# Extract datetime components
df['date_only'] = df['date'].dt.date
df['time_only'] = df['date'].dt.time
df['day_of_week'] = df['date'].dt.day_name()
df['hour'] = df['date'].dt.hour
# Create time categories
time_labels = ['Late Night (12AM-6AM)', 'Morning (6AM-12PM)', 
               'Afternoon (12PM-6PM)', 'Evening (6PM-12AM)']
df['time_category'] = pd.cut(
    df['hour'],
    bins=[0, 6, 12, 18, 24],
    labels=time_labels,
    right=False
)

# Clean up columns
df.drop(columns=['hour', 'entities'], inplace=True)

In [15]:
# Final Data Structure and Export
# Select and order final columns
final_columns = [
    # Metadata
    'channel', 'date', 'date_only', 'time_only', 
    'day_of_week', 'time_category', 'sender_id', 
    'views', 'media_type',
    
    # Content
    'message', 'clean_message', 'tokens',
    
    # Entities
    'brands', 'attributes', 'materials'
]

processed_df = df[final_columns]

# Save to CSV
processed_df.to_csv('../data/cleaned_processed_data.csv', index=False)

# Save to JSON
processed_df.to_json('../data/cleaned_processed_dat.json', orient='records', force_ascii=False)

print("Processing complete. Data saved to CSV and JSON files.")

Processing complete. Data saved to CSV and JSON files.
